## 다중 분류를 이용한 포유류, 조류분류(텐서플로 1.x)

- 285 ~ 310

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
from sklearn import datasets
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
tf.enable_eager_execution()
tf.__version__

'1.15.2'

In [ ]:
# 훈련 데이터
X = np.array([
              [0, 0],
              [1, 0],
              [1, 1],
              [0, 0],
              [0, 1],
], dtype = 'float32')

# 정답 데이터
y = np.array([
              [1, 0, 0],
              [0, 1, 0],
              [0, 0, 1],
              [1, 0, 0],
              [0, 0, 1],
], dtype = 'float32')

In [ ]:
# cost가 0으로 수렴하는데 적절한 w,b 선택해줌
initializer = tf.contrib.layers.xavier_initializer()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
# weight
w0 = tf.Variable(initializer([2, 10]))

# bias
b0 = tf.Variable(initializer([10]))

# weight
w1 = tf.Variable(initializer([10, 3]))

# bias
b1 = tf.Variable(initializer([3]))

In [ ]:
# 1차 예측
# 예측 
hypothesis0 = tf.matmul(X, w0) + b0

# 예측값을 0~1 사이의 값으로 변환
hypothesis0  = tf.sigmoid(hypothesis0)

# 2차 예측
# 예측 
hypothesis1 = tf.matmul(hypothesis0, w1) + b1

# 활성화 함수 softmax
# softmax 함수 -> 확률로 나타내는데, 진짜인거 같은것에 높은 확률을 줌
hypothesis1 = tf.nn.softmax(hypothesis1)

print(hypothesis1)

tf.Tensor(
[[0.21499588 0.09673163 0.6882724 ]
 [0.21138947 0.10209084 0.68651974]
 [0.21949922 0.0960512  0.6844496 ]
 [0.21499588 0.09673163 0.6882724 ]
 [0.2258333  0.09332795 0.6808388 ]], shape=(5, 3), dtype=float32)


In [ ]:
optimizer = tf.train.AdamOptimizer(0.01)

In [ ]:
for step in range(1001):
  with tf.GradientTape() as tape:
    hypothesis0 = tf.sigmoid(tf.matmul(X, w0)+b0)

    hypothesis1 = tf.nn.softmax(tf.matmul(hypothesis0, w1)+ b1)

    cost = -tf.reduce_mean(y * tf.log(hypothesis1))

    grads = tape.gradient(cost, [w0, w1, b0, b1])

    optimizer.apply_gradients(grads_and_vars = zip(grads, [w0, w1, b0, b1]))

    if step % 100 ==0:
      print("=" * 50)
      print("step: {}, cost:{}".format(step, cost.numpy()))
      print("="*50)

step: 0, cost:0.40798234939575195
step: 100, cost:0.11125652492046356
step: 200, cost:0.02056073024868965
step: 300, cost:0.007721001747995615
step: 400, cost:0.004094456788152456
step: 500, cost:0.002564608585089445
step: 600, cost:0.0017687507206574082
step: 700, cost:0.0012985155917704105
step: 800, cost:0.0009958649752661586
step: 900, cost:0.00078880216460675
step: 1000, cost:0.0006404047016985714


In [ ]:
# 예측
hypothesis0 = tf.sigmoid(tf.matmul(X, w0) +b0)

predict = tf.nn.softmax(tf.matmul(hypothesis0, w1) +b1)

# 각 줄의 최댓값 인덱스 리턴
predict01 = tf.argmax(predict,1)

# 정답값의 최댓값 인덱스 리턴
target = tf.argmax(y, 1)

# 두개가 동일한지 비교 -> True, Flase
ac01 = tf.equal(predict01,target)

# True -> 1, False -> 0으로 변환
ac02 = tf.cast(ac01, dtype = "float32")

# 정확도
ac03 = tf.reduce_mean(ac02)
print("정확도 : ",ac03.numpy())

정확도 :  1.0


## 다중 분류를 이용한 포유류, 조류 분류(텐서플로 2.x)

- 302 ~ 312

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
# 훈련 데이터
X = np.array([
              [0, 0],
              [1, 0],
              [1, 1],
              [0, 0],
              [0, 1],
], dtype = 'float32')

# 정답 데이터
y = np.array([
              [1, 0, 0],
              [0, 1, 0],
              [0, 0, 1],
              [1, 0, 0],
              [0, 0, 1],
], dtype = 'float32')

In [ ]:
# 객체 생성
model = Sequential()

# 히든레이어 1층
# 입력 칸수 2, 출력 칸수 10
model.add(Dense(10, input_dim = 2, activation = 'sigmoid'))

# 출력 칸수 3
# 선형회귀 후 softmax 사용
model.add(Dense(3, activation = 'softmax'))

model.summary()

# 'categorical_crossentropy' : -(y * tf.log(hypothesis1)) 합이 최소가 되는 w,b찾기
model.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.01), metrics = ['acc'])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 10)                30        
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 33        
Total params: 63
Trainable params: 63
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs = 1000)

Epoch 1/1000
1/1 [==============================] - 1s 702ms/step - loss: 1.3055 - acc: 0.2000
Epoch 2/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.2671 - acc: 0.2000
Epoch 3/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.2328 - acc: 0.2000
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 1.2026 - acc: 0.2000
Epoch 5/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.1760 - acc: 0.0000e+00
Epoch 6/1000
1/1 [==============================] - 0s 7ms/step - loss: 1.1527 - acc: 0.4000
Epoch 7/1000
1/1 [==============================] - 0s 6ms/step - loss: 1.1325 - acc: 0.4000
Epoch 8/1000
1/1 [==============================] - 0s 14ms/step - loss: 1.1149 - acc: 0.4000
Epoch 9/1000
1/1 [==============================] - 0s 10ms/step - loss: 1.0998 - acc: 0.4000
Epoch 10/1000
1/1 [==============================] - 0s 13ms/step - loss: 1.0867 - acc: 0.4000
Epoch 11/1000
1/1 [==============================] - 0s 9m

In [ ]:
predict = model.predict(X)

# 각 줄의 최댓값 인덱스 리턴
predict01 = np.argmax(predict,1)

# 정답값의 최댓값 인덱스 리턴
target = tf.argmax(y, 1)

# 두개가 동일한지 비교 -> True, Flase
predict02 = (predict01 == target)

# 정확도
ac03 = np.sum(predict02)/len(predict02)
print("정확도 : ",ac03)

정확도 :  1.0
